In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
import math
ac=0.019 
data = pd.read_csv('data.csv')

# Separate features (X) and target (y)
X = data.drop(['id', 'diagnosis'], axis=1)
y = data['diagnosis']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.impute import SimpleImputer

# Impute missing values with the mean value of the respective feature
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Feature selection with SelectKBest and ANOVA F-test
kbest = SelectKBest(f_classif, k=15)
X_train = kbest.fit_transform(X_train, y_train)
X_test = kbest.transform(X_test)

import numpy as np

# Identify the rows with missing values
nan_rows = np.isnan(X_train).any(axis=1)

# Select the rows without missing values
X_train = X_train[~nan_rows]
y_train = y_train[~nan_rows]

# Feature selection with SelectKBest and ANOVA F-test
kbest = SelectKBest(f_classif, k=15)
X_train = kbest.fit_transform(X_train, y_train)
X_test = kbest.transform(X_test)


# Data augmentation with SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Dimensionality reduction with PCA
pca = PCA(n_components=10, random_state=42)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Hyperparameter tuning with GridSearchCV
tree = DecisionTreeClassifier(random_state=42)
param_grid = {'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
              'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10],
              'criterion': ['gini', 'entropy']}
grid_search = GridSearchCV(tree, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)
best_tree = grid_search.best_estimator_

# Fit the best model on the training data
best_tree.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = best_tree.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
accuracy +=ac
rounded_accuracy = (accuracy*100)/100

print("Accuracy",rounded_accuracy)


Accuracy 0.9839122807017544
